Como hemos comentado en el artículo al que acompaña este notebook, vamos a crear un modelo de TensorFlow para resolver el caso de uso de fraude con tarjetas de crédito.

Primero importamos las librerías requeridas:

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

El primer paso es importar el dataset y separarlo en conjuntos de train, test y val:

In [7]:
# Load the dataset
df = pd.read_csv('creditcard.csv')

# Split the dataset into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['Class', 'Time'], axis=1), df['Class'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

Tenemos que definir las métricas que vamos a calcular con la red neuronal:

In [8]:
# Define the metrics for the neural network
precision = tf.keras.metrics.Precision(name='precision')
recall = tf.keras.metrics.Recall(name='recall')
auc = tf.keras.metrics.AUC(name='auc')

Ya podemos definir la red neuronal. La explicación de cada capa se incluye en el artículo:

In [9]:
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

La red neuronal se tiene que compilar antes de poder lanzar el entrenamiento:

In [12]:
# Compile the model with binary crossentropy loss and class weights
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precision, recall, auc])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Train the model
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    callbacks=[callback]
                   )

Epoch 1/20
334/334 [==============================] - 5s 10ms/step - loss: 0.0059 - accuracy: 0.9993 - precision: 0.8421 - recall: 0.6957 - auc: 0.9311 - val_loss: 0.0034 - val_accuracy: 0.9996 - val_precision: 0.8841 - val_recall: 0.7821 - val_auc: 0.9293
Epoch 2/20
334/334 [==============================] - 3s 10ms/step - loss: 0.0048 - accuracy: 0.9992 - precision: 0.8485 - recall: 0.7089 - auc: 0.9413 - val_loss: 0.0026 - val_accuracy: 0.9995 - val_precision: 0.8514 - val_recall: 0.8077 - val_auc: 0.9346
Epoch 3/20
334/334 [==============================] - 3s 8ms/step - loss: 0.0037 - accuracy: 0.9993 - precision: 0.8534 - recall: 0.7184 - auc: 0.9559 - val_loss: 0.0027 - val_accuracy: 0.9995 - val_precision: 0.8939 - val_recall: 0.7564 - val_auc: 0.9402
Epoch 4/20
334/334 [==============================] - 3s 8ms/step - loss: 0.0049 - accuracy: 0.9993 - precision: 0.8789 - recall: 0.7120 - auc: 0.9397 - val_loss: 0.0028 - val_accuracy: 0.9996 - val_precision: 0.8649 - val_recall:

Finalmente, evaluaremos el modelo:

In [13]:
# Evaluate the model
test_loss, test_acc, test_prec, test_rec, test_auc = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)
print('Test Precision:', test_prec)
print('Test Recall:', test_rec)
print('Test AUC:', test_auc)

1781/1781 [==============================] - 4s 2ms/step - loss: 0.0033 - accuracy: 0.9993 - precision: 0.8571 - recall: 0.7347 - auc: 0.9421
Test Loss: 0.0032608122564852238
Test Accuracy: 0.9993329048156738
Test Precision: 0.8571428656578064
Test Recall: 0.7346938848495483
Test AUC: 0.9420897960662842


Como mencionamos en el artículo, incluimos también un modelo Random Forest y un modelo XGBoost:

In [14]:
# RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight={0: 1, 1: 50})
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf.predict(X_test)

# Compute precision, recall, F1-score, and AUC
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1score_rf = f1_score(y_test, y_pred_rf, average='weighted')
auc_rf = roc_auc_score(y_test, y_pred_rf)

# XGBClassifier
xgb = XGBClassifier(n_estimators=100, random_state=42, scale_pos_weight=50)
xgb.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = xgb.predict(X_test)

# Compute precision, recall, F1-score, and AUC
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1score_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
auc_xgb = roc_auc_score(y_test, y_pred_xgb)

# Print the results
print('Random Forest Classifier:')
print('Precision:', precision_rf)
print('Recall:', recall_rf)
print('F1-score:', f1score_rf)
print('AUC:', auc_rf)

print('\nXGBoost Classifier:')
print('Precision:', precision_xgb)
print('Recall:', recall_xgb)
print('F1-score:', f1score_xgb)
print('AUC:', auc_xgb)

Random Forest Classifier:
Precision: 0.9736842105263158
Recall: 0.7551020408163265
F1-score: 0.9995147437962991
AUC: 0.8775334345893674

XGBoost Classifier:
Precision: 0.963855421686747
Recall: 0.8163265306122449
F1-score: 0.9996160812287971
AUC: 0.9081368865779289
